In [1]:
import os

# Set Java 17 for PySpark
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home'
os.environ['PATH'] = '/opt/homebrew/opt/openjdk@17/bin:' + os.environ.get('PATH', '')

from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.appName("Spark SQL Query Dataframes").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/12 22:27:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data_path = '/Users/zlatavorobeva/Documents/GitHub/SQL Int/SQL_Intermediate/Spark/Data'

In [3]:
json_df2_path = data_path + "/utilization.json"
df = spark.read.format("json").load(json_df2_path)

In [4]:
df.show(10)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
+---------------+-------------------+-

In [5]:
df.createOrReplaceTempView("utilization")

In [6]:
df_count = spark.sql("SELECT count(*) FROM utilization")
df_count.show()

+--------+
|count(1)|
+--------+
|  500000|
+--------+



In [7]:
df_sql = spark.sql("SELECT count(*) \
                    FROM utilization \
                    WHERE session_count > 70")
df_sql.show()

+--------+
|count(1)|
+--------+
|  239659|
+--------+



In [8]:
df_sql = spark.sql("SELECT server_id, count(*) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id")
df_sql.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      103|    8744|
|      104|    7366|
|      100|     391|
|      105|    1110|
|      101|    9808|
|      102|    8586|
|      112|    7425|
|      113|    9418|
|      110|    2826|
|      107|    5646|
|      111|    3093|
|      108|    8375|
|      109|    3129|
|      116|    1167|
|      114|    2128|
|      115|    5284|
|      120|    2733|
|      118|    7913|
|      117|    3605|
|      126|    6365|
+---------+--------+
only showing top 20 rows


In [9]:
df_sql = spark.sql("SELECT server_id, count(*) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      101|    9808|
|      113|    9418|
|      145|    9304|
|      103|    8744|
|      102|    8586|
|      133|    8583|
|      108|    8375|
|      149|    8288|
|      137|    8248|
|      148|    8027|
|      123|    7918|
|      118|    7913|
|      112|    7425|
|      139|    7383|
|      104|    7366|
|      121|    7084|
|      142|    7084|
|      146|    7072|
|      126|    6365|
|      144|    6220|
+---------+--------+
only showing top 20 rows


In [10]:
df_sql = spark.sql("SELECT server_id, min(session_count), avg(session_count), max(session_count) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show()

+---------+------------------+------------------+------------------+
|server_id|min(session_count)|avg(session_count)|max(session_count)|
+---------+------------------+------------------+------------------+
|      101|                71| 87.66557911908646|               105|
|      113|                71| 86.96262476109577|               103|
|      145|                71| 86.97732158211522|               103|
|      103|                71| 85.76372369624886|               101|
|      102|                71| 85.71150710458886|               101|
|      133|                71| 85.46720260981009|               100|
|      108|                71|  85.1219104477612|               100|
|      149|                71|  84.9612693050193|                99|
|      137|                71|  85.0061833171678|                99|
|      148|                71| 84.70350068518749|                99|
|      123|                71| 84.53220510229856|                98|
|      118|                71| 84.

In [11]:
df_sql = spark.sql("SELECT server_id, min(session_count), round(avg(session_count),2), max(session_count) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show()

+---------+------------------+----------------------------+------------------+
|server_id|min(session_count)|round(avg(session_count), 2)|max(session_count)|
+---------+------------------+----------------------------+------------------+
|      101|                71|                       87.67|               105|
|      113|                71|                       86.96|               103|
|      145|                71|                       86.98|               103|
|      103|                71|                       85.76|               101|
|      102|                71|                       85.71|               101|
|      133|                71|                       85.47|               100|
|      108|                71|                       85.12|               100|
|      149|                71|                       84.96|                99|
|      137|                71|                       85.01|                99|
|      148|                71|                      

25/12/11 07:56:24 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 957319 ms exceeds timeout 120000 ms
25/12/11 07:56:24 WARN SparkContext: Killing executors is not supported by current scheduler.
25/12/11 07:56:31 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:669)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1296)
	at o